## Time Series Data Generation
This involves the generation of the data for time series analysis results for the given EEG data. The current analysis involves the EDF files of 6 different patients. The number of sampling points vary for the data as the machine used for the analysis varies and the condition in which the data was acquired also varies.

Using the pandas library the data in RAW EDF files can be analysed further to give the CSV files with the data of interest. 

### Spherical Splines to remove bad channels in EEG data

The mapping of the Scalp potential (SP) and the Scalp current densities (SCD) are better and accurate enough when the data is approximated using Spherical splines rather than plate splines. 

This involves the analysis of the statistical quantities of the EEG signal. 
https://github.com/mne-tools/mne-python/blob/master/examples/preprocessing/plot_interpolate_bad_channels.py
https://www.sciencedirect.com/science/article/pii/S1474442202000030#bib40

---
#### REFERENCES:
-  Perrin, F., Pernier, J., Bertrand, O. and Echallier, JF. (1989)
   Spherical splines for scalp potential and current density mapping.
   Electroencephalography and Clinical Neurophysiology, Feb; 72(2):184-7.
- [EDF Files Specification](https://www.edfplus.info/specs/edf.html)
- [SET Files Specification]()

In [1]:
import mne
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


mne.set_log_level('WARNING')

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline

In [5]:
ch_names = ['Fpz', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'F8', 'T4', 'T6', 'A2', 'O2', 'C4', 'P4']
file_names = []
file_csv_names = []

def read_edf():
    '''
    Reads the edf files from the data folder
    '''
    path = os.getcwd() + '\\data\\'
    files = []
    print("Reading files...")
    for f in os.listdir(path):
        global file_names
        files.append(mne.io.read_raw_edf(path+f, preload=True))
        file_names.append(f)
        print(f"Reading {f} now..")
    print('Reading successfully completed!')
    return files


def get_info(files):
    '''
    Gets the info from the patients and displays it
    '''
    for index, item in enumerate(files):
        print(f"> Record Information of patient {index+1}:\n{item.info}")
        print("-"*40)
    print('Information generated!')
    return True

def set_montage(files, montage):
    for index, f in enumerate(files):
        print(f"Setting montage for patient {index+1}")
        files[index].set_montage(montage)
        print(f"Patient {index+1}")
        print("Sampling frequency")
        print(f.info["sfreq"])
        print("LPF freq:")
        print(f.info["lowpass"])
        print(f"Length (s): {f._data[-1].shape[0]/f.info['sfreq']}") # get the data length
        print('-'*20)
    return ("Setting montage completed!")
            
def to_csv(files):
    global file_csv_names
    global file_names
    for index, f in enumerate(files):
        print(files[index]._data.shape) # 36 rows each corresponding to a channel from the required list of channels
        df = pd.DataFrame()
        dictOfChannels = { i : ch_names[i] for i in range(0, len(ch_names) ) }
        print(f'appending channels to the dataframe {file_names[index]}')
        for index1, channel in enumerate(files[index].info["ch_names"]):
            if channel in ch_names:
                curr = files[index]._data[:][index1]
                df = df.append(pd.Series(curr), ignore_index= True)
                
        df = df.transpose()
        df.rename(columns=dictOfChannels, inplace=True)
        print("*"*20)
        for f in file_names:
              file_csv_names.append(f[:-4].strip()+'.csv')
        path = os.getcwd() + '\\csv_clean\\'
        print(f"Writing CSV file to {path+file_csv_names[index]}")
        df.to_csv(path+file_csv_names[index], header = True)
        print('CSV written')

### Reading files

In [6]:
files = read_edf()

Reading files...
Reading AB sz1.edf now..
Reading AI sz1.edf now..
Reading AI sz2.edf now..
Reading CR sz1.edf now..
Reading CR sz2.edf now..
Reading CR sz3.edf now..
Reading DM sz1.edf now..
Reading JM sz1.edf now..
Reading JM sz2.edf now..
Reading JM sz3.edf now..
Reading JM sz4.edf now..
Reading JM sz5.edf now..
Reading KS sz1.edf now..
Reading successfully completed!


### Writing the columns under consideration in CSV files

In [7]:
to_csv(files)

(36, 99200)
appending channels to the dataframe AB sz1.edf
********************
Writing CSV file to C:\Users\Lingesh K\Desktop\EpilepticSeizurePrediction\csv_clean\AB sz1.csv
CSV written
(36, 130400)
appending channels to the dataframe AI sz1.edf
********************
Writing CSV file to C:\Users\Lingesh K\Desktop\EpilepticSeizurePrediction\csv_clean\AI sz1.csv
CSV written
(36, 307200)
appending channels to the dataframe AI sz2.edf
********************
Writing CSV file to C:\Users\Lingesh K\Desktop\EpilepticSeizurePrediction\csv_clean\AI sz2.csv
CSV written
(36, 31200)
appending channels to the dataframe CR sz1.edf
********************
Writing CSV file to C:\Users\Lingesh K\Desktop\EpilepticSeizurePrediction\csv_clean\CR sz1.csv
CSV written
(36, 59200)
appending channels to the dataframe CR sz2.edf
********************
Writing CSV file to C:\Users\Lingesh K\Desktop\EpilepticSeizurePrediction\csv_clean\CR sz2.csv
CSV written
(36, 73600)
appending channels to the dataframe CR sz3.edf
****

### Checking the metadata in an EDF file

In [ ]:
files[0].info # Prints the info of all the meta data in one edf file

### Setting the montage for the EDF file

In [ ]:
# Set the montage to the standard 10-20 system
montage = mne.channels.read_montage(kind='standard_1020', ch_names=ch_names)
set_montage(files, montage) 

In [ ]:
len(files[0].info["dig"]) # Gives the details of the digitization of the channels

In [ ]:
files[0].set_eeg_reference(ref_channels=['Ref']) # Set the reference channel for the file AB sz1.edf

### Analysing the Maximum and Minimum value of first patient EDF file

In [ ]:
curr = files[0]._data
num = curr.shape[0]
print(f"Number of channels for patient 1: {num}")
# plotting the maximum and minimum of each channel
for i in range(num):
    print(f"channel {i+1}: {files[0].info['ch_names'][i]}")
    print(f"Minimum : int({min(curr[i])*1e+6}) Maximum: int({max(curr[i])*1e+6})")

### Reading an EDF file and Converting to CSV file 
\['Event',
 'Ref',
 'X1',
 'X2',
 'X3',
 'X4',
 'X5',
 'X6',
 'X7',
 'X8',
 'X9',
 'X10',
 'OSAT',
 'Status'\] are the channels that are ignored for the ICA procedure. This gives a 22 component result of weighted electrodes. From this, the best 13 channels are chosen and the data is taken for analysis 

In [67]:
src_path = os.getcwd() + '\\Denoised Data\\denoised.edf'
file = mne.io.read_raw_edf(src_path, preload=True)
print(file._data.shape)
df = pd.DataFrame()
ch_names = ['Fpz', 'Fz', 'Cz', 'Pz', 'Fp2', 'F4', 'F8', 'T4', 'T6', 'A2', 'O2', 'C4', 'P4']
dictOfChannels = { i : ch_names[i] for i in range(0, len(ch_names) ) }
for index1, channel in enumerate(file.info["ch_names"]):
    if channel in ch_names:
        curr = file._data[:][index1]
        df = df.append(pd.Series(curr), ignore_index= True)
df = df.transpose()
df.rename(columns=dictOfChannels, inplace=True)
df.to_csv(os.getcwd()+'\\Denoised Data\\denoised.csv', header=True)
print('CSV Written')

(36, 307200)
CSV Written
